In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [5]:
import word2vec

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zhongyifan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 28))



KeyboardInterrupt: 

In [ ]:
word2vec.word2phrase('text8', 'text8-phrases', verbose=True)

In [ ]:
word2vec.word2vec('text8-phrases', 'text8.bin', size=100, verbose=True)


In [ ]:
word2vec.word2clusters('text8', 'text8-clusters.txt', 100, verbose=True)

### Predictions

In [7]:
model = word2vec.load('text8.bin')

In [9]:
model.vectors.shape

(98331, 100)

In [28]:
model_words = model.vocab.tolist()

In [10]:
model['dog'][:10]

array([ 0.04115792,  0.06849291,  0.06320069,  0.04258114,  0.07664289,
       -0.05450514,  0.01255086,  0.21808557,  0.08820264, -0.06714589])

#### Find the words similar to 'socks'

In [19]:
indexes, metrics = model.cosine('surreal')
indexes, metrics

(array([25359, 18174, 21970,  7940, 34370, 37240, 20633, 42417, 25490,
        83603]),
 array([0.81881375, 0.79293867, 0.78502176, 0.78031298, 0.7792365 ,
        0.77237696, 0.77006564, 0.7675051 , 0.76462544, 0.7632895 ]))

In [20]:
model.generate_response(indexes, metrics)

rec.array([('fairy_tale', 0.81881375), ('witty', 0.79293867),
           ('suspense', 0.78502176), ('vampire', 0.78031298),
           ('blaxploitation', 0.7792365 ), ('parodying', 0.77237696),
           ('fables', 0.77006564), ('charmed', 0.7675051 ),
           ('pastiche', 0.76462544), ('patter', 0.7632895 )],
          dtype=[('word', '<U78'), ('metric', '<f8')])

In [21]:
model.generate_response(indexes, metrics).tolist()

[('fairy_tale', 0.8188137456710571),
 ('witty', 0.7929386656140519),
 ('suspense', 0.7850217619070058),
 ('vampire', 0.7803129800356738),
 ('blaxploitation', 0.7792365020450462),
 ('parodying', 0.7723769563053509),
 ('fables', 0.7700656422464125),
 ('charmed', 0.7675050965033734),
 ('pastiche', 0.7646254395967957),
 ('patter', 0.763289495876089)]

### Phrases

In [ ]:
indexes, metrics = model.cosine('los_angeles')
model.generate_response(indexes, metrics).tolist()

### Analogies

In [23]:
indexes, metrics = model.analogy(pos=['king', 'woman'], neg=['man'], n=10)
indexes, metrics

(array([1087, 1145, 7523, 3141, 1335,  648, 1826, 8419, 6768, 1426]),
 array([0.28923294, 0.27296378, 0.26543243, 0.26422935, 0.26240602,
        0.25964164, 0.25902549, 0.25892429, 0.25788254, 0.25618148]))

In [24]:
model.generate_response(indexes, metrics).tolist()

[('queen', 0.28923293556852936),
 ('prince', 0.2729637771281497),
 ('empress', 0.26543243292197893),
 ('monarch', 0.2642293548891466),
 ('wife', 0.2624060230724133),
 ('emperor', 0.2596416371806385),
 ('throne', 0.25902548539952636),
 ('aragon', 0.25892429360600044),
 ('regent', 0.2578825381757367),
 ('bishop', 0.25618148308820166)]

### Train word2vec on IMDB Reviews

In [6]:
from __future__ import print_function
from os.path import dirname
import os,sys,inspect

current_dir = dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
manage_dir = dirname(dirname(dirname(current_dir)))

if not manage_dir in sys.path:
    sys.path.insert(0, manage_dir)

# from pyml4.common import context, db
# context.local_context.print_summary()

import gensim
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import logging
import nltk
import nltk.data
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import webtext
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.corpus import stopwords
import pymysql.cursors

In [4]:
logger = logging.getLogger(__name__)
sent_detector = None

# store NLTK resources inside ML4
#nltk_download_dir = os.path.dirname(nltk.__file__)
#nltk.data.path.append(os.path.dirname(nltk.__file__))
#nltk.download('punkt', download_dir=os.path.dirname(nltk.__file__))
nltk.download('punkt')
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

NUM_REVIEWS_TO_FETCH = 1000000

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zhongyifan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
class ReviewSentences(object):
    # def __init__(self):
    #     self.db_client = db.manually_connect()
    #
    # def __del__(self):
    #     self.db_client.close()

    def __iter__(self):
        connection = pymysql.connect(host="127.0.0.1",
                                    user="web",
                                    password="atth1132",
                                    db="ML3_mirror")

        with connection.cursor() as cur:
        #with self.db_client.cursor() as cur:
        #cur.execute("select imdbReviewId, movieId, body, upVotes, downVotes from imdb_review ")
            cur.execute("""select imdbReviewId, movieId, body, upVotes, downVotes
                            FROM (select imdbReviewId, movieId, body, upVotes, downVotes,
                            @rn:= if(@movieId = movieId, @rn+1, 1) as rn,
                            @movieId:=movieId as pi
                            from imdb_review, (select @rn:=0) b
                            order by movieId, (upVotes-downVotes)desc) x
                            where x.rn <= 80 and x.pi < 5500
                            """)
            for (review_id, movie_id, body, up, down) in cur:
                for sentence in sent_detector.tokenize(body):
                    yield (movie_id, sentence)

In [6]:
review_sentences = [(movie_id,sentence) for (movie_id,sentence) in ReviewSentences()]
#review_sentences

In [9]:
#Remove punctuations
#nltk.download('stopwords')
tokenizer = RegexpTokenizer(r'\w+')

stop_words = set(stopwords.words('english'))
print(stop_words)

{'against', 'itself', 'their', 'themselves', 'to', 'me', 'isn', "didn't", 'our', 'then', 'you', 'that', 'do', 'after', "won't", "wasn't", "it's", 'by', "mightn't", 'won', 'have', 'does', 'of', 'herself', 'before', 'there', 'too', 'she', 'during', 'how', 'in', 'from', 'own', "shouldn't", 'be', 'hers', 'below', "mustn't", 'has', 'its', 'himself', 'being', 'because', 'and', 'when', 'o', 'y', 'each', 'if', 'more', 'than', 'his', 'once', 'any', 'about', 'hadn', "wouldn't", 'll', 'yourself', 'up', 'at', 'not', 'it', 'down', 'all', 'few', 'further', 'off', 'had', 'hasn', 'theirs', 'haven', 'but', 'we', 'your', 'shouldn', 've', 'what', 'both', 'can', 'are', 'over', 're', 'out', "that'll", 'weren', 'so', 'on', 'ourselves', 'they', 'these', 's', "should've", 'them', 'don', 'were', 'doing', "hasn't", 'needn', 'a', 'for', 'whom', "doesn't", 'couldn', 'i', 'or', 'my', 'some', 'should', 't', 'is', 'yours', 'he', 'again', 'only', 'will', "you're", 'most', 'd', 'did', "aren't", 'same', "don't", 'doesn

In [10]:
stemmer = SnowballStemmer("english")
lemma = WordNetLemmatizer()

# stem and remove stopwords from reviews
review_sent = []
only_sentence = []
review_word = []
#
nltk.download('wordnet')

for i in review_sentences:
    #print("Hi")
    #print(i)
    word_lst = [lemma.lemmatize(word,pos='v') for word in tokenizer.tokenize(i[1].lower()) if (word not in stop_words)]
    review_word.append((i[0],word_lst))
    sent = ' '.join(word_lst)
    review_sent.append((i[0],sent))
    only_sentence.append(sent)
print('finish lemmatizing')
#



[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zhongyifan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
finish lemmatizing


In [11]:
sentence_list = []
for movieId, sentence in review_word:
    sentence_list.append(sentence)
    

In [12]:
#len(only_sentence_list)
len(sentence_list)

9801201

In [ ]:
df = pd.DataFrame(sentence_list, columns=["sentence"])

In [ ]:
df.to_csv('word2vec_sentence_list.csv', index=False)

In [60]:
# print('start training model1')
# word_vec_model1 = gensim.models.Word2Vec(sentence_list, alpha=0.025, size=100, min_alpha=0.025,min_count=5,workers=35,iter=20)
# #word_vec_model1.train(only_sentence,total_examples=word_vec_model1.corpus_count,epochs=word_vec_model1.iter)
# word_vec_model1.save('Word2Vec_model2')
# print('first model trained')

#Model2 size 1000
print('start training model2')
word_vec_model1 = gensim.models.Word2Vec(sentence_list, alpha=0.025, size=1000, min_alpha=0.025,min_count=5,workers=35,iter=20)
#word_vec_model1.train(only_sentence,total_examples=word_vec_model1.corpus_count,epochs=word_vec_model1.iter)
word_vec_model1.save('Word2Vec_model2')
print('first model trained')

start training


KeyboardInterrupt: 

In [7]:
# Model 1 trained on 100 movie reviews
word_vec_model1 = Word2Vec.load('Word2Vec_model1')

In [8]:
len(word_vec_model1.wv.vocab)

480

In [10]:
#word_vec_model1.wv.__getitem__('idea')
word_vec_model1.wv.similar_by_word("idea")

[('laugh', 0.9966999292373657),
 ('warm', 0.9961812496185303),
 ('screen', 0.9959357380867004),
 ('appeal', 0.9957133531570435),
 ('interest', 0.9956430792808533),
 ('humour', 0.9953981041908264),
 ('also', 0.9949686527252197),
 ('funny', 0.9947707653045654),
 ('development', 0.9947367906570435),
 ('later', 0.994699239730835)]